In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import os
from tqdm import tqdm

In [3]:
num_classes = 32
num_epochs = 20
batch_size = 2048
input_size = 1
model_dir = 'model'
window_size = 10
num_layers = 2
hidden_size = 64
file_dir = 'data_supervised'

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def generate_dataset(file_dir):
    sessions = []
    normal_data = set()
    labels = []
    max_len = 0
    with open(file_dir+'/normal', 'r') as f:
        for ln in f.readlines():
            ln = [0] + list(map(lambda n: n, map(int, ln.strip().split()))) + [30]
            max_len = max(max_len,len(ln))
            normal_data.add(tuple(ln))
    abnormal_data = set()
    with open(file_dir+'/abnormal', 'r') as f:
        for ln in f.readlines():
            ln = [0] + list(map(lambda n: n, map(int, ln.strip().split()))) + [30]
            max_len = max(max_len,len(ln))
            abnormal_data.add(tuple(ln))
    
    for line in tqdm(normal_data, "normal:"):
        sessions.append(line)
        labels.append(0)
    for line in tqdm(abnormal_data, "normal:"):
        sessions.append(line)
        labels.append(1)
    print('Number of sessions({}): {}'.format(file_dir, len(sessions)))
    print('Number of normal sessions: {}'.format(len(normal_data)))
    print('Number of abnormal sessions: {}'.format(len(abnormal_data)))
    sessions = TensorDataset(torch.tensor(sessions, dtype=torch.float), torch.tensor(labels))

    # print('Number of sessions({}): {}'.format(name, len(hdfs)))
    return sessions

In [8]:
dataset = generate_dataset(file_dir)

normal:: 100%|█████████████████████████████████████████████████████████████████| 4123/4123 [00:00<00:00, 458435.80it/s]

Number of sessions(data_supervised): 4962
Number of normal sessions: 839
Number of abnormal sessions: 4123


ValueError: expected sequence of length 22 at dim 1 (got 28)